<img src="images/picto_transparent.png" alt="CARS logo" width="100" align="right"/>

# Dense dsm with stereo-matching methods comparison

This notebook analyses the contribution of two different stereomatching methods available in Cars 



Please, let's see the sensor_to_dense_dsm_step_by_step notebook before this one

### Imports

In [ ]:
# Notebook local imports

import os
import math
###
# Silent OTB info logs
os.environ['OTB_LOGGER_LEVEL']='WARNING'
import warnings
# Filter warnings
warnings.filterwarnings("ignore",category=UserWarning)
# import pretty print
import pprint 
pp = pprint.PrettyPrinter(indent=4)

# import external function for notebook
from notebook_helpers import get_full_data, show_data, get_dir_path, set_up_demo_inputs



In [ ]:
# CARS imports

# Applications
from cars.applications.application import Application
from cars.applications.grid_generation import grid_correction
from cars.applications.sparse_matching import sparse_matching_tools

# Pipelines
import cars.pipelines.sensor_to_dense_dsm.sensor_dense_dsm_constants as sens_cst
from cars.pipelines.sensor_to_dense_dsm import sensors_inputs
from cars.pipelines.sensor_to_dense_dsm import dsm_output

# Conf, core, orchestrator
from cars.core import cars_logging
from cars.core import inputs, preprocessing
from cars.core.utils import safe_makedirs
from cars.orchestrator import orchestrator
from cars.core.utils import make_relative_path_absolute

In [ ]:
# Show CARS version
from cars import __version__
print("CARS version used : {}".format(__version__))

---------

## Inputs/Outputs

### Define outputs

In [ ]:
# Modify with your own output path if needed
output_dir = os.path.join(get_dir_path(), "output_notebook")
print(output_dir)

### Define inputs

In [ ]:
# By default, the notebook use data_gizeh_small.tar.bz2, data_gizeh is available also (performance dependent).
# For you own data: Modify input_dir_path and modify all images, geometric models and color file names below
input_dir_path = set_up_demo_inputs("data_gizeh_small")

inputs_conf = {
    "sensors": {
        "left": {
            "image": os.path.join(input_dir_path, "img1.tif"),
            "geomodel": {
              "path": os.path.join(input_dir_path, "img1.geom")
            },
            "color": os.path.join(input_dir_path, "color1.tif"),
            "no_data": 0,


        },
        "right": {
            "image": os.path.join(input_dir_path, "img2.tif"),
            "geomodel": {
              "path": os.path.join(input_dir_path, "img2.geom")
            },
            "no_data": 0,
        },   
    },
    "pairing": [["left", "right"]],
    "initial_elevation": os.path.join(input_dir_path, "srtm_dir")
}

inputs = sensors_inputs.sensors_check_inputs(inputs_conf)
pp.pprint(inputs)

In [ ]:
# Get geometry plugin
(
    _,
    geom_plugin_without_dem_and_geoid,
    geom_plugin_with_dem_and_geoid,
) = sensors_inputs.check_geometry_plugin(
    None, inputs
)

### Create orchestrator


In [ ]:
# Use sequential mode in notebook
orchestrator_conf = {"mode": "sequential"}
cars_orchestrator = orchestrator.Orchestrator(orchestrator_conf=orchestrator_conf, out_dir=output_dir)

---------

## First, compute epipolar rectified images 

In [ ]:
epipolar_grid_generation_application = Application("grid_generation")

In [ ]:
resampling_application = Application("resampling")

In [ ]:
sparse_matching_application = Application("sparse_matching")

### Sensors images generation

From input configuration "inputs" seen before

In [ ]:
_, sensor_image_left, sensor_image_right = sensors_inputs.generate_inputs(inputs, geom_plugin_without_dem_and_geoid)[0]

### Grid Generation : epipolar grid generation

In [ ]:
grid_left, grid_right = epipolar_grid_generation_application.run(
    sensor_image_left,
    sensor_image_right,
    geom_plugin_with_dem_and_geoid,
    orchestrator=cars_orchestrator
)

### Resampling : epipolar images generation


In [ ]:
epipolar_image_left, epipolar_image_right = resampling_application.run(
    sensor_image_left,
    sensor_image_right,
    grid_left,
    grid_right,
    orchestrator=cars_orchestrator,
    margins=sparse_matching_application.get_margins()
)

###  Sparse matching: compute sifts

In [ ]:
epipolar_matches_left, _ = sparse_matching_application.run(
    epipolar_image_left,
    epipolar_image_right,
    grid_left.attributes["disp_to_alt_ratio"],
    orchestrator=cars_orchestrator
)

### Grid correction: correct epipolar grids from sparse matches
Find correction to apply, and generate new right epipolar grid

#### Filter matches

In [ ]:
matches_array = sparse_matching_application.filter_matches(epipolar_matches_left, orchestrator=cars_orchestrator)

#### Estimate grid correction

In [ ]:
grid_correction_coef, corrected_matches_array,_, _, _ = grid_correction.estimate_right_grid_correction(matches_array, grid_right)

#### Correct right grid

In [ ]:
corrected_grid_right = grid_correction.correct_grid(grid_right, grid_correction_coef, False, output_dir)

### Estimate disp min and disp max from sparse matches

#### Compute margins used in dense matching, with corresponding disparity min and max

In [ ]:
triangulation_application = triangulation_application = Application("triangulation")

In [ ]:
dense_matching_application = Application("dense_matching")

In [ ]:
dmin, dmax = sparse_matching_tools.compute_disp_min_disp_max(
                    sensor_image_left,
                    sensor_image_right,
                    grid_left,
                    corrected_grid_right,
                    corrected_matches_array,
                    orchestrator=cars_orchestrator,
                    disp_margin=(
                        sparse_matching_application.get_disparity_margin()
                    ),
                    disp_to_alt_ratio=grid_left.attributes["disp_to_alt_ratio"],
                    geometry_plugin=geom_plugin_with_dem_and_geoid
)

In [ ]:
dense_matching_margins, disp_min, disp_max = dense_matching_application.get_margins(
    grid_left, disp_min=dmin, disp_max=dmax)

###  Resampling: generate epipolar images with corrected grids and new margins

In [ ]:
new_epipolar_image_left, new_epipolar_image_right = resampling_application.run(
    sensor_image_left,
    sensor_image_right,
    grid_left,
    corrected_grid_right,
    orchestrator=cars_orchestrator,
    margins=dense_matching_margins,
    optimum_tile_size=(
        dense_matching_application.get_optimal_tile_size(
            disp_min, 
            disp_max,
            cars_orchestrator.cluster.checked_conf_cluster[
                "max_ram_per_worker"
            ]
        )
    ),
    add_color=True,
)

## Dense Matching: compute disparities with pandora by using two differents methods

### Census similarity measure with semi-global matching (default method)


In [ ]:
dense_matching_census_application = Application("dense_matching")

In [ ]:
epipolar_disparity_map_census = dense_matching_census_application.run(
    new_epipolar_image_left,
    new_epipolar_image_right,
    orchestrator=cars_orchestrator,
    disp_min=disp_min,
    disp_max=disp_max,
)

#### Show full disparity map

In [ ]:
data_disparity_census = get_full_data(epipolar_disparity_map_census, "disp")
show_data(data_disparity_census)

### MC-CNN, the similarity measure produced by mc-cnn neural network

<a href="https://github.com/CNES/Pandora_MCCNN">MC-CNN algorithm</a> used by <a  href="https://github.com/CNES/Pandora"> Pandora</a> as <a href="https://github.com/CNES/Pandora_plugin_mccnn">plugin</a>

In [ ]:
dense_matching_mccnn_application = Application("dense_matching", cfg={"method": "mccnn_sgm"})

In [ ]:
epipolar_disparity_map_mccnn = dense_matching_mccnn_application.run(
    new_epipolar_image_left,
    new_epipolar_image_right,
    orchestrator=cars_orchestrator,
    disp_min=disp_min,
    disp_max=disp_max,
)

In [ ]:
data_disparity_mccnn = get_full_data(epipolar_disparity_map_mccnn, "disp")
show_data(data_disparity_mccnn)

## Compute two DSM and compare them

One  from disparity map computed by Census similarity measure and the other from disparity map from MC-CNN similarity measure

In [ ]:
conf_outlier_removing_small_components = {"method": "small_components", "activated": True}
pc_outlier_removing_small_comp_application = Application("point_cloud_outliers_removing", cfg=conf_outlier_removing_small_components)

In [ ]:
conf_outlier_removing_small_statistical = {"method": "statistical", "activated": True}
pc_outlier_removing_stats_application = Application("point_cloud_outliers_removing", cfg=conf_outlier_removing_small_statistical)

In [ ]:
pc_fusion_application = Application("point_cloud_fusion")

In [ ]:
conf_rasterization = { 
    "method": "simple_gaussian",
    "dsm_radius": 3,
    "sigma": 0.3
}
rasterization_application = Application("point_cloud_rasterization", cfg=conf_rasterization)

 Compute epsg

In [ ]:
epsg = preprocessing.compute_epsg(
    sensor_image_left, 
    sensor_image_right,
    grid_left,
    corrected_grid_right,
    geom_plugin_with_dem_and_geoid,
    orchestrator=cars_orchestrator,
    disp_min=disp_min,
    disp_max=disp_max
)

### Triangulation : triangulate matches

From census disparity map

In [ ]:
epipolar_points_cloud_census = triangulation_application.run(
    sensor_image_left,
    sensor_image_right,
    new_epipolar_image_left,
    grid_left,
    corrected_grid_right,
    epipolar_disparity_map_census,
    epsg,
    geom_plugin_without_dem_and_geoid,
    orchestrator=cars_orchestrator,
    uncorrected_grid_right=grid_right,
    geoid_path=inputs[sens_cst.GEOID],
    disp_min=disp_min,
    disp_max=disp_max,
)

From mccnn disparity map

In [ ]:
epipolar_points_cloud_mccnn = triangulation_application.run(
    sensor_image_left,
    sensor_image_right,
    new_epipolar_image_left,
    grid_left,
    corrected_grid_right,
    epipolar_disparity_map_mccnn,
    epsg,
    geom_plugin_without_dem_and_geoid,
    orchestrator=cars_orchestrator,
    uncorrected_grid_right=grid_right,
    geoid_path=inputs[sens_cst.GEOID],
    disp_min=disp_min,
    disp_max=disp_max,
)

 #### Compute terrain bounding box

In [ ]:
current_terrain_roi_bbox = preprocessing.compute_terrain_bbox(
    sensor_image_left,
    sensor_image_right,
    new_epipolar_image_left,
    grid_left,
    corrected_grid_right,
    epsg,
    geom_plugin_with_dem_and_geoid,
    resolution=rasterization_application.get_resolution(),
    disp_min=disp_min,
    disp_max=disp_max,
    orchestrator=cars_orchestrator
)
terrain_bounds, optimal_terrain_tile_width = preprocessing.compute_terrain_bounds(
    [current_terrain_roi_bbox],
    resolution=rasterization_application.get_resolution()
)

#### Transform point cloud to terrain point cloud

From census disparity map

In [ ]:
merged_points_clouds_census = pc_fusion_application.run(
    [epipolar_points_cloud_census],
    terrain_bounds,
    epsg,
    orchestrator=cars_orchestrator,
    margins=rasterization_application.get_margins(),
    optimal_terrain_tile_width=optimal_terrain_tile_width
)

From mccnn disparity map

In [ ]:
merged_points_clouds_mccnn = pc_fusion_application.run(
    [epipolar_points_cloud_mccnn],
    terrain_bounds,
    epsg,
    orchestrator=cars_orchestrator,
    margins=rasterization_application.get_margins(),
    optimal_terrain_tile_width=optimal_terrain_tile_width
)

### Point Cloud Outlier Removing : remove points with small components removing method

From census disparity map

In [ ]:
filtered_sc_merged_points_clouds_census = pc_outlier_removing_small_comp_application.run(
    merged_points_clouds_census,
    orchestrator=cars_orchestrator,
)

From mccnn disparity map

In [ ]:
filtered_sc_merged_points_clouds_mccnn = pc_outlier_removing_small_comp_application.run(
    merged_points_clouds_mccnn,
    orchestrator=cars_orchestrator,
)

### Point Cloud Outlier Removing: remove points with statistical removing method

From census disparity map

In [ ]:
filtered_stats_merged_points_clouds_census = pc_outlier_removing_stats_application.run(
    filtered_sc_merged_points_clouds_census,
    orchestrator=cars_orchestrator,
)

From mccnn disparity map

In [ ]:
filtered_stats_merged_points_clouds_mccnn = pc_outlier_removing_stats_application.run(
    filtered_sc_merged_points_clouds_mccnn,
    orchestrator=cars_orchestrator,
)

### Rasterization : rasterize point cloud

From census disparity map

In [ ]:
dsm_census = rasterization_application.run(
    filtered_stats_merged_points_clouds_census,
    epsg,
    orchestrator=cars_orchestrator
)

From mccnn disparity map

In [ ]:
dsm_mccnn = rasterization_application.run(
    filtered_stats_merged_points_clouds_mccnn,
    epsg,
    orchestrator=cars_orchestrator
)

### Show DSM


From census disparity map

In [ ]:
data_dsm_census = get_full_data(dsm_census, "hgt")
show_data(data_dsm_census, mode="dsm")

From mccnn disparity map

In [ ]:
data_dsm_mccnn = get_full_data(dsm_mccnn, "hgt")
show_data(data_dsm_mccnn, mode="dsm")

### Show ortho image

From census disparity map

In [ ]:
data_ortho_census = get_full_data(dsm_census, "img")
show_data(data_ortho_census, mode='image')


From mccnn disparity map

In [ ]:
data_ortho_mccnn = get_full_data(dsm_mccnn, "img")
show_data(data_dsm_mccnn, mode='image')
